As the main goal of this challenge, you are expected to create a hash function based on quantum simulation. Your hash function's performance evaluation will be based on the following criteria:

1. Output determinism
2. Preservation of entropy
3. Computational difficulty
4. Preimage resistance
5. Collision resistance
6. Computational feasibility
7. Computation time
8. Purely quantum hashing

Their meaning will be demonstrated on a simple (and very bad) hash function.

The following hash function uses one qubit per one byte of input and applies the RX gates with the angles proportional to the bytes' values:

In [16]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, Statevector
import numpy as np

def simple_quantum_hash(input_bytes: bytes):
    num_qubits = len(input_bytes)
    qc = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        angle = (input_bytes[i] / 255) * np.pi  # scale to [0, π]
        qc.rx(angle, i)
     
    sv = Statevector.from_instruction(qc)
    exp_vals = [sv.expectation_value(Pauli("Z"), [i]).real for i in range(num_qubits)]

    # Map each expectation value from [-1, 1] to an 8-bit integer in [0, 255].
    output_bytes = bytearray([min(int(((val + 1) / 2) * 256), 255) for val in exp_vals])
    
    return output_bytes

At the very least, this function meets 2 of our most straightforward requirements. Firstly, it consistently produces the same output for a given input, satisfying the 'Output determinism' constraint, and, secondly, it does not use any classical hashing algorithms as required by the 'Purely classical hashing' point.

Let's now see what output our hash function produces for some simple inputs:

In [17]:
print(list(simple_quantum_hash(bytes(range(0, 260, 20)))))

[255, 252, 240, 222, 198, 170, 139, 108, 78, 50, 28, 11, 2]


As you might've already deduced from the function definition, it basically calculates shifted and quantized cosine value for each of the input bytes. Unfortunately this fails to pass the 'Computational difficulty' requirement, as it is trivial to implement the same function without using any quantum simulators and such function would run in linear time with respect to the input length. This also makes finding the preimage of a given hash output an easy task, making it a cryptographically-poor hash function that is non-compliant with the 'Preimage resistance' criteria.

Since the cosine itself is a bijection from the domain of [0, π] to the codomain of [-1, 1] the hash collisions can only be possible due to the quantization. The easiest way to find such collisions is to look at the values closest to the minimum and maximum of a single byte range, where the derivative of the cosine is small:

In [20]:
print(list(simple_quantum_hash(bytes(range(0, 20)))))
print(list(simple_quantum_hash(bytes(range(236, 256)))))

[255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 254, 254, 254, 254, 253, 253, 253, 252, 252]
[3, 3, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


As you can see, there is plenty of byte collisions to be found in those regions. As a result, using this hash algorithm would result in a higher collision rate when compared to its classical counterparts, making it unsuitable for production applications and failing the 'Collision resistance' constraint.

The last non-trivial requirement is the 'Preservation of entropy', which our function, yet again, does not pass. The reason for that is simple - the cosine is not a linear function. That means that feeding a large set of randomly-generated inputs to our function would result in its output bytes being more highly concentrated towards their maximum and minimum values (as was previously demonstrated) and failing to behave like a sample from a random distribution over the function's codomain. Furthermore, some of the byte values are not possible to obtain at all due to the quantization:

In [21]:
print(list(simple_quantum_hash(bytes(range(120, 135)))))

[139, 138, 136, 135, 133, 131, 130, 128, 127, 125, 124, 122, 120, 119, 117]


Lastly, the provided hash function does not pass the 'Computational feasibility' requirement, since it would require to simulate a 32-qubit circuit to calculate a 256-bit hash, which is not feasible to do on general-purpose hardware. This also makes the 'Computation time' criteria irrelevant, even thogh the function only employs one gate per qubit which would lead to good performance if the number of qubits was decreased.